# Video Games Category - EDA

**Category**: Video Games

**Team Member**: Yaiechnyk Oleh

## 0) Setup

In [1]:
import os
from pyspark.sql import functions as F, Window
from amazon_reviews_analysis.utils import build_spark

spark = build_spark()
print("Spark version:", spark.version)


openjdk version "17.0.15" 2025-04-15
OpenJDK Runtime Environment Temurin-17.0.15+6 (build 17.0.15+6)
OpenJDK 64-Bit Server VM Temurin-17.0.15+6 (build 17.0.15+6, mixed mode, sharing)



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/11 01:59:27 WARN Utils: Your hostname, MacBook-Pro-Oleh.local, resolves to a loopback address: 127.0.0.1; using 192.168.31.157 instead (on interface en0)
25/11/11 01:59:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 01:59:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1


In [2]:
RAW_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../data/raw"))
META_PATH = os.path.join(RAW_DIR, "meta_categories",  "meta_Video_Games.jsonl")
REV_PATH  = os.path.join(RAW_DIR, "review_categories", "Video_Games.jsonl")

print("Meta path exists:", os.path.exists(META_PATH), META_PATH)
print("Rev  path exists:", os.path.exists(REV_PATH),  REV_PATH)

meta_raw = spark.read.json(META_PATH)
rev_raw  = spark.read.json(REV_PATH)

Meta path exists: True /Users/olehyaiechnyk/PycharmProjects/amazon-reviews-analysis/data/raw/meta_categories/meta_Video_Games.jsonl
Rev  path exists: True /Users/olehyaiechnyk/PycharmProjects/amazon-reviews-analysis/data/raw/review_categories/Video_Games.jsonl


## 1) Data Health — nulls & empties

In [3]:

def null_report(df, name):
    total = df.count()
    rows = []
    for c in df.columns:
        n = df.where(F.col(c).isNull()).count()
        rows.append((c, n, n/total if total else 0.0))
    out = spark.createDataFrame(rows, ["column","null_count","null_rate"])
    print(f"--- Nulls in {name} (total {total}) ---")
    out.orderBy(F.desc("null_rate")).show(50, truncate=False)
    return out

meta_nulls = null_report(meta_raw, "metadata")
rev_nulls  = null_report(rev_raw, "reviews")


25/11/11 01:59:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


--- Nulls in metadata (total 137269) ---


+---------------+----------+--------------------+
|column         |null_count|null_rate           |
+---------------+----------+--------------------+
|bought_together|137269    |1.0                 |
|author         |137007    |0.998091338903904   |
|subtitle       |136919    |0.9974502618945282  |
|price          |75261     |0.5482738273025957  |
|main_category  |11035     |0.08038959998251609 |
|store          |4361      |0.031769736794177855|
|title          |0         |0.0                 |
|rating_number  |0         |0.0                 |
|videos         |0         |0.0                 |
|average_rating |0         |0.0                 |
|description    |0         |0.0                 |
|features       |0         |0.0                 |
|details        |0         |0.0                 |
|categories     |0         |0.0                 |
|images         |0         |0.0                 |
|parent_asin    |0         |0.0                 |
+---------------+----------+--------------------+


--- Nulls in reviews (total 4624615) ---
+-----------------+----------+---------+
|column           |null_count|null_rate|
+-----------------+----------+---------+
|timestamp        |0         |0.0      |
|images           |0         |0.0      |
|helpful_vote     |0         |0.0      |
|user_id          |0         |0.0      |
|text             |0         |0.0      |
|verified_purchase|0         |0.0      |
|title            |0         |0.0      |
|parent_asin      |0         |0.0      |
|asin             |0         |0.0      |
|rating           |0         |0.0      |
+-----------------+----------+---------+



Metadata (137,269 products): Shows significant missing data. bought_together (100%), author (99.8%), and subtitle (99.7%) are almost entirely null. Crucially, price is null for 54.8% of products.

Reviews (4,624,615 reviews): The dataset is perfectly clean in terms of nulls. All columns report 0 nulls and a 0.0 null rate.

In [4]:
array_like_cols = [c for c, t in meta_raw.dtypes if t.startswith("array") or t.startswith("map")]

def empty_rate_expr(colname):
    return F.avg(F.when(F.col(colname).isNull() | (F.size(F.col(colname)) == 0), 1).otherwise(0)).alias(colname)

if array_like_cols:
    empties = meta_raw.select([empty_rate_expr(c) for c in array_like_cols]).collect()[0].asDict()
    print("--- Empty rate (metadata array/map-like) ---")
    for k, v in empties.items():
        print(f"{k}: {float(v):.4f}")
else:
    print("No array/map fields detected.")


--- Empty rate (metadata array/map-like) ---
categories: 0.0921
description: 0.3769
features: 0.2877
images: 0.0010
videos: 0.7894


The output shows that videos are empty 78.9% of the time, description is empty 37.7%, and features is empty 28.8%. This indicates that while not "null," these fields often lack data.

## 2) Deduplicate & trim reviews

In [5]:
dup_keys = ["user_id","asin","timestamp"]
rev_counts_before = rev_raw.count()
rev_dups = (rev_raw.groupBy([F.col(c) for c in dup_keys]).count().filter("count > 1"))
print("Duplicate groups:", rev_dups.count())

w = Window.partitionBy(*dup_keys).orderBy(F.lit(1))
rev_dedup = rev_raw.withColumn("rn", F.row_number().over(w)).where("rn = 1").drop("rn")
print("After dedup:", rev_dedup.count())

# Remove empty/whitespace-only text
rev_clean = rev_dedup.where(F.length(F.trim(F.col("text"))) > 0)
print("Removed empty texts:", rev_dedup.count() - rev_clean.count())
print("Final reviews:", rev_clean.count())


Duplicate groups: 45196


After dedup: 4570967


Removed empty texts: 3564


Final reviews: 4567403


This step removes 3,564 empty-text reviews, resulting in a final, clean review dataset (rev_clean) containing 4,567,403 reviews.

## 3) Price cleaning

In [6]:
from pyspark.sql import functions as F

num_pat = r'([0-9]{1,3}(?:,[0-9]{3})*(?:\.[0-9]+)?|[0-9]+(?:\.[0-9]+)?)'

meta_price = (
    meta_raw
      .withColumn("price_raw", F.col("price").cast("string"))
      .withColumn("price_num_str", F.regexp_extract(F.col("price_raw"), num_pat, 1))
      .withColumn(
          "price_num_str",
          F.when(F.length("price_num_str") == 0, F.lit(None))
           .otherwise(F.regexp_replace(F.col("price_num_str"), ",", ""))
      )
      .withColumn("clean_price", F.col("price_num_str").cast("double"))
)

total_products  = meta_price.count()
nonnull_prices  = meta_price.filter(F.col("clean_price").isNotNull()).count()
print(f"Price coverage: {nonnull_prices}/{total_products} = {nonnull_prices/total_products:.2%}")

(meta_price
  .filter(F.col("clean_price").isNotNull())
  .select(
      F.mean("clean_price").alias("mean_price"),
      F.stddev("clean_price").alias("std_price"),
      F.expr("percentile(clean_price, array(0.5))")[0].alias("median_price"),
      F.min("clean_price").alias("min_price"),
      F.max("clean_price").alias("max_price"),
  )
  .show(truncate=False)
)


Price coverage: 61992/137269 = 45.16%


+---------------+-----------------+------------+---------+---------+
|mean_price     |std_price        |median_price|min_price|max_price|
+---------------+-----------------+------------+---------+---------+
|44.743041844107|69.55919557995735|24.95       |0.0      |999.99   |
+---------------+-----------------+------------+---------+---------+



## 4) Platform parsing

In [7]:
import re
@F.udf("string")
def parse_platform(title, categories):
    base = ""
    if title:
        base += " " + title.lower()
    if categories is not None:
        try:
            if isinstance(categories, list):
                base += " " + " ".join([(" ".join(x) if isinstance(x, list) else str(x)) for x in categories])
            else:
                base += " " + str(categories)
        except Exception:
            pass
    txt = base.lower()
    patterns = [
        ("PlayStation 5", r"\bps5\b|\bplaystation 5\b"),
        ("PlayStation 4", r"\bps4\b|\bplaystation 4\b"),
        ("PlayStation 3", r"\bps3\b|\bplaystation 3\b"),
        ("Xbox Series",   r"\bxbox series\b|\bxbox series x\b|\bxbox series s\b"),
        ("Xbox One",      r"\bxbox one\b"),
        ("Xbox 360",      r"\bxbox 360\b"),
        ("Nintendo Switch", r"\bswitch\b|\bnintendo switch\b"),
        ("Wii U",         r"\bwii u\b"),
        ("Wii",           r"\bwii\b"),
        ("3DS/DS",        r"\b3ds\b|\b2ds\b|\bds\b"),
        ("PC",            r"\bpc\b|\bwindows\b|\bsteam\b|\bmac\b"),
    ]
    for label, pat in patterns:
        if re.search(pat, txt):
            return label
    return "Other/Unknown"

meta_clean = (meta_price
              .withColumn("platform", parse_platform(F.col("title"), F.col("categories")))
             )

meta_clean.groupBy("platform").count().orderBy(F.desc("count")).show(50, truncate=False)


+---------------+-----+
|platform       |count|
+---------------+-----+
|PC             |35700|
|Other/Unknown  |28077|
|Nintendo Switch|17323|
|PlayStation 4  |14157|
|3DS/DS         |8488 |
|Xbox One       |7388 |
|PlayStation 3  |5820 |
|Xbox 360       |5683 |
|PlayStation 5  |5301 |
|Wii            |5292 |
|Wii U          |2132 |
|Xbox Series    |1908 |
+---------------+-----+



The table shows us the distribution of the 137,269 products by the newly parsed platform. PC is the most common (35,700), followed by Nintendo Switch (17,323) and PlayStation 4 (14,157).

## 5) Linkage check

In [8]:

rev_pa = rev_clean.select("parent_asin").distinct().withColumnRenamed("parent_asin","pa_r")
meta_pa = meta_clean.select("parent_asin").distinct().withColumnRenamed("parent_asin","pa_m")

missing = rev_pa.join(meta_pa, rev_pa.pa_r == meta_pa.pa_m, "left_anti").count()
print("review parent_asin missing in metadata:", missing)
print("distinct parent_asin in reviews:", rev_pa.count())
print("distinct parent_asin in metadata:", meta_pa.count())


review parent_asin missing in metadata: 0


distinct parent_asin in reviews: 137196
distinct parent_asin in metadata: 137269


## 6) Сleaned datasets

In [9]:
from pathlib import Path

PROJ_ROOT = Path.cwd().parents[1]
OUT_DIR = PROJ_ROOT / "data" / "raw" / "review_categories"
OUT_DIR.mkdir(parents=True, exist_ok=True)

REV_OUT = OUT_DIR / "reviews_clean.parquet"
META_OUT = OUT_DIR / "metadata_clean.parquet"

rev_clean.write.mode("overwrite").parquet(str(REV_OUT))
meta_clean.write.mode("overwrite").parquet(str(META_OUT))


print("✓ Saved:")
print(" -", REV_OUT)
print(" -", META_OUT)


✓ Saved:
 - /Users/olehyaiechnyk/PycharmProjects/amazon-reviews-analysis/data/raw/review_categories/reviews_clean.parquet
 - /Users/olehyaiechnyk/PycharmProjects/amazon-reviews-analysis/data/raw/review_categories/metadata_clean.parquet
